In [1]:
%matplotlib inline
import collections
import pandas as pd
from io import StringIO

In [2]:
testlines = '''32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483'''

In [3]:
with open('day7input.txt') as fp:
    data = fp.read()

## Part 1

In [4]:
cardnums = {card:hex(num+2)[-1] for num, card in enumerate('23456789TJQKA')}

In [5]:
cardnums

{'2': '2',
 '3': '3',
 '4': '4',
 '5': '5',
 '6': '6',
 '7': '7',
 '8': '8',
 '9': '9',
 'T': 'a',
 'J': 'b',
 'Q': 'c',
 'K': 'd',
 'A': 'e'}

In [6]:
typenums = {type: str(num+1) for num,type in enumerate(['High card', 'One pair', 'Two pair', 'Three of a kind',
                                                'Full house', 'Four of a kind', 'Five of a kind'])}

In [7]:
typenums

{'High card': '1',
 'One pair': '2',
 'Two pair': '3',
 'Three of a kind': '4',
 'Full house': '5',
 'Four of a kind': '6',
 'Five of a kind': '7'}

In [8]:
def get_type(hand):
    c = collections.Counter(hand)
    maxcnt = max(c.values())
    if maxcnt == 5:
        return 'Five of a kind'
    elif maxcnt == 4:
        return 'Four of a kind'
    elif maxcnt == 3:
        # either three of a kind, or a full house
        if 2 in c.values():
            return 'Full house'
        else:
            return 'Three of a kind'
    elif maxcnt == 2:
        # either one pair, or two pair
        cv = collections.Counter(c.values())
        if cv[2] == 2:
            return 'Two pair'
        else:
            return 'One pair'
    else:
        return 'High card'

In [9]:
def get_score(hand):
    type = get_type(hand)
    s = [typenums[type]]
    for c in hand:
        s.append(cardnums[c])
    return int(f'0x{"".join(s)}', 16)

In [10]:
get_score('32T3K')

2304573

In [11]:
def part1(inputstr, returndf=False):
    df = pd.read_table(StringIO(inputstr), sep=' ', names=['hand', 'bid'])
    df['type'] = df['hand'].apply(get_type)
    df['score'] = df['hand'].apply(get_score)
    dfsort = df.sort_values('score', ascending=True, ignore_index=True)
    dfsort['rank'] = dfsort.index + 1
    dfsort['winnings'] = dfsort['bid']*dfsort['rank']
    if returndf:
        return dfsort
    return dfsort['winnings'].sum()

In [12]:
part1(testlines, True)

,hand,bid,type,score,rank,winnings
0,32T3K,765,One pair,2304573,1,765
1,KTJJT,220,Two pair,4041658,2,440
2,KK677,28,Two pair,4052599,3,84
3,T55J5,684,Three of a kind,4871605,4,2736
4,QQQJA,483,Three of a kind,5033150,5,2415


In [13]:
part1(testlines)

6440

In [14]:
part1(data)

248396258

## Part 2

In [15]:
cardnums2 = {card:hex(num+1)[-1] for num, card in enumerate('J23456789TQKA')}

In [16]:
cardnums2

{'J': '1',
 '2': '2',
 '3': '3',
 '4': '4',
 '5': '5',
 '6': '6',
 '7': '7',
 '8': '8',
 '9': '9',
 'T': 'a',
 'Q': 'b',
 'K': 'c',
 'A': 'd'}

In [17]:
def get_score2(hand):
    type = get_best_type(hand)
    s = [typenums[type]]
    for c in hand:
        s.append(cardnums2[c])
    return int(f'0x{"".join(s)}', 16)

In [18]:
def get_best_type(hand):
    if 'J' not in hand:
        return get_type(hand)
    if all(c == 'J' for c in hand):
        return 'Five of a kind'
    best_type = 'High card'
    for c in set(hand) - set('J'):
        h = hand.replace('J', c)
        t = get_type(h)
        if typenums[t] > typenums[best_type]:
            best_type = t
    return best_type
    

In [19]:
get_best_type('T55J5'), get_best_type('JJJJJ')

('Four of a kind', 'Five of a kind')

In [20]:
for line in testlines.split('\n'):
    hand = line.split()[0]
    print(hand, get_best_type(hand))

32T3K One pair
T55J5 Four of a kind
KK677 Two pair
KTJJT Four of a kind
QQQJA Four of a kind


In [21]:
def part2(inputstr, returndf=False):
    df = pd.read_table(StringIO(inputstr), sep=' ', names=['hand', 'bid'])
    df['type'] = df['hand'].apply(get_best_type)
    df['score'] = df['hand'].apply(get_score2)
    dfsort = df.sort_values('score', ascending=True, ignore_index=True)
    dfsort['rank'] = dfsort.index + 1
    dfsort['winnings'] = dfsort['bid']*dfsort['rank']
    if returndf:
        return dfsort
    return dfsort['winnings'].sum()

In [22]:
part2(testlines, True)

,hand,bid,type,score,rank,winnings
0,32T3K,765,One pair,2304572,1,765
1,KK677,28,Two pair,3982967,2,56
2,T55J5,684,Four of a kind,6968597,3,2052
3,QQQJA,483,Four of a kind,7060253,4,1932
4,KTJJT,220,Four of a kind,7119130,5,1100


In [23]:
part2(testlines)

5905

In [24]:
part2(data)

246436046